<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

<div class="alert alert-block" style="background-color: #c7b8d6; padding: 10px">
    <p style="color: #222222">
        <b>Note:</b>
        <br>
        This Jupyter notebook uses hardware features of the Zynq UltraScale+ RFSoC device. Therefore, the notebook cells will only execute successfully on an RFSoC platform.
    </p>
</div>

# Notebook Set A

---

## 04 - Overlays and Hardware Interfacing
This notebook will expand your understanding of PYNQ overlays by investigating a simple overlay design on RFSoC. The overlay design consists of a Numerically Controlled Oscillator (NCO) that generates a cosine and sine wave in the RFSoC's Programmable Logic (PL). We will use the PYNQ Overlay class, DefaultIP class, and DefaultHierarchy class to design drivers for the NCO system. Finally, we will perform visualisation and analysis of the time domain signal and corresponding frequency spectra. Widgets will be used for the purpose of changing the NCO operating parameters during run-time.

## Table of Contents
* [1. Introduction](#introduction)
* [2. The PYNQ-NCO Overlay](#overlays)
    * [2.1. The DefaultIP Class](#default-ip-class)
    * [2.2. The DefaultHierarchy Class](#default-hierarchy)
* [3. Numerically Controlled Oscillator](#nco-example)
    * [3.1. NCO Properties](#nco-properties)
    * [3.2. Widgets](#widgets)
* [4. Visualisation and Analysis](#visualisation-and-analysis)
    * [4.1. Time Plot](#time-plot)
    * [4.2. Frequency Spectra](#frequency-spectra)
* [5. Conclusion](#conclusion)

## References
* [1] - [StrathSDR, "PYNQ NCO Overlay", GitHub Repository.](https://github.com/strath-sdr/pynq_nco)
* [2] - [AMD, "Read the Docs: PYNQ Overlays", v3.0.0](https://pynq.readthedocs.io/en/v3.0.0/pynq_overlays.html)
* [3] - [AMD, "Read the Docs: pynq.overlay.Overlay", v3.0.0](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.overlay.html#pynq.overlay.Overlay)
* [4] - [AMD, "Read the Docs: pynq.overlay.DefaultIP", v3.0.0](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.overlay.html#pynq.overlay.DefaultIP)
* [5] - [AMD, "Read the Docs: pynq.overlay.DefaultHierarchy", v3.0.0](https://pynq.readthedocs.io/en/v3.0.0/pynq_package/pynq.overlay.html#pynq.overlay.DefaultHierarchy)
* [6] - [Plotly, "Plotly Website", webpage.](https://plotly.com/)
* [7] - [jupyter-widgets, “GitHub Source Code Repository for the IPywidgets Python Library,” webpage.](https://github.com/jupyter-widgets/ipywidgets)

## Revision
* **v1.0** | 23/01/23 | *First Revision*

---


## 1. Introduction <a class="anchor" id="introduction"></a>
We will use a custom Python module known as PYNQ-NCO [1] extensively in this notebook. The PYNQ-NCO module was developed by the StrathSDR team and contains software code and hardware libraries that implement a Numerically Controlled Oscillator (NCO). NCO's are capable of generating cosine and sine waves using primitive logic elements, such as those that reside in the RFSoC's PL. The PYNQ-NCO example in this notebook will demonstrate how to interact with a PL design using built-in PYNQ classes [2-5] and the visualisation library plotly [6] and interactive widgets library ipywidgets [7].

The PYNQ-NCO module was installed automatically when you installed the RFSoC-Book notebooks on your RFSoC platform. See the [PYNQ-NCO GitHub repository](https://github.com/strath-sdr/pynq_nco) if you would like to explore the associated software code and hardware integration libraries on your own.

## 2. The PYNQ-NCO Overlay <a class="anchor" id="overlays"></a>
We will begin by importing the PYNQ-NCO overlay class, named `NumericalOverlay()`. This class inherits methods and properties from the PYNQ overlay class [3]. During initialisation, the PYNQ overlay class discovers Intellectual Property (IP) Cores and hierarchies that reside in the FPGA design. Drivers are then bound to the IP Cores and hierarchies using PYNQ's built-in DefaultIP [4] and DefaultHierarchy [5] classes, or a user's custom class.

In [ ]:
from pynq_nco.overlay import NumericalOverlay

ol = NumericalOverlay()

We can inspect the contents of the design in the FPGA logic fabric of your RFSoC device. The overlay class contains an IP dictionary, which has a description of each hardware accelerator, or IP Core, in the design. This dictionary was created during initialisation. Descriptions of IP Cores were added to the design as they were discovered by the overlay class.

In [ ]:
ol.ip_dict

The PYNQ-NCO example contains an NCO IP Core, and a Data Inspector hierarchy block (more on hierarchies in a moment). The Data Inspector consists of two IP Cores known as a Packet Generator and Advanced eXtensible Interface (AXI) Direct Memory Access (DMA). Figure 1 presents the IP integration design for the FPGA logic fabric.

<figure>
<img src="./images/nco_ip_integrator.png" style="width: 100%;"/>
    <figcaption><b>Figure 1: Vivado IP Integrator design for the PYNQ-NCO system.</b></figcaption>
</figure>

Each IP Core and hierarchy have been assigned a software driver that is responsible for its control and status. These drivers inherit methods and properties from PYNQ's DefaultIP and DefaultHierarchy classes. We will discuss these further below.

### 2.1. The DefaultIP Class <a class="anchor" id="default-ip-class"></a>
The DefaultIP class is assigned to IP Cores that do not have a more specific driver already assigned. The DefaultIP class provides the user with register read and write capabilities for an IP Core (with an AXI4-Lite interface). The user may choose to inherit the DefaultIP class to build a more specific IP Core driver. For instance, the NCO IP Core has its own class based on the DefaultIP class, which is responsible for configuring the NCO frequency and gain. The methods and properties relating to the software driver for the NCO IP Core can be found running the cell below.

In [ ]:
help(ol.nco)

Notice that the NumericalOscillator class has four methods, two data descriptors, and several inherited methods and properties from the PYNQ DefaultIP class. The DefaultIP class has provided register read and write methods. These have been used to design NCO specific methods and properties that communicate with the FPGA design. Notable custom methods include:

```python
ol.nco.complex_enable() # Enables the Cosine and Sine wave output of the NCO
ol.nco.real_enable()    # Enables the Cosine wave output only of the NCO
ol.nco.disable()        # Disables the Cosine and Sine wave output of the NCO
```

There are also custom properties:

```python
ol.nco.frequency        # The output frequency of the NCO
ol.nco.gain             # The output gain of the NCO
```

We will use these methods and properties shortly when configuring the output of the NCO. You can inspect the driver source code for the NCO [here](https://github.com/strath-sdr/pynq_nco/blob/main/pynq_nco/nco.py).

### 2.2. The DefaultHierarchy Class <a class="anchor" id="default-hierarchy"></a>
Hierarchies are a collection of IP Cores (and/or other hierarchies) that have been grouped together. In the PYNQ-NCO design, there is one hierarchy that is responsible for transferring NCO data from the FPGA to the PS, so that we can plot and visualise its contents. This hierarchy is named the `data_inspector` and it has its own custom driver that inherits the DefaultHierarchy class.

The DefaultHierarchy class can be used as the base class for a user to design custom drivers for their own hierarchy. The `data_inspector` hierarchy in the PYNQ-NCO design inherits many useful tools from the DefaultHierarchy including parsers, descriptors, and Partial Reconfiguration (PR) related objects, methods, and properties. We can discover more about the `data_inspector` hierarchy by running the following cell.

In [ ]:
help(ol.data_inspector)

Many of the methods and properties above are related to the DefaultHierarchy class. The `data_inspector` has one significant custom method for moving data between the PL and PS, given below.

```python
ol.data_inspector.transfer(packetsize)
"""Transfer samples from the FPGA logic fabric to the Processing System.
         
    Parameters
    ----------
        packetsize : int
            number of samples to transfer
"""
```

The custom method above will be used later to inspect Cosine and Sine waves generated from the NCO.

## 3. Numerically Controlled Oscillator <a class="anchor" id="nco-example"></a>
In the analogue world, oscillators are implemented using feedback loops with resonators (often quartz crystal), which control the frequency of oscillations. These oscillators are useful when defining clock signals and sine waveforms. In the digital domain, an equivalent oscillator can be constructed, which is often refferred to as a Numerically Controlled Oscillator, or NCO. NCO's can be implemented using various techniques. A common way is to store samples of a cosine or sine wave in Read-Only Memory (ROM) and read each sample out at a rate corresponding to the desired output frequency.

An NCO implemented using the ROM method has several features including a phase input, an accumulator (phase generator) and a ROM for storing values (often implemented using FPGA Look-Up Tables). See the left of Figure 2 for a simplified diagram of this NCO design.

<figure>
<img src="./images/nco_block_diagram.png" style="width: 80%;"/>
    <figcaption><b>Figure 2: Functional block diagram of the NCO design.</b></figcaption>
</figure>

Also included on the right side of the diagram is the packet generator and AXI Datamover (or AXI DMA) that are present in the `data_inspector` hierarchy. These IP Cores are used to transfer samples of data from the NCO into Jupyter Labs for visualisation.

### 3.1. NCO Properties <a class="anchor" id="nco-properties"></a>
The NCO architecture above has already been designed and implemented on your RFSoC platform. It is currently operating in the FPGA portion of your RFSoC device. The software driver we defined previously can be used to interact with the input phase and gain stage of the design. Firstly, let us configure the desired frequency of the NCO using the `ol.nco.frequency` property, which appropriately configures the input phase.

In [ ]:
ol.nco.frequency = 12e6 # Hz

Now, we can also configure the gain of the NCO's output wave.

In [ ]:
ol.nco.gain = 0.5 # Value between -1 and 1

Lastly, we can either set the NCO to output a cosine wave only, or a cosine and sine wave at the same time. Later we will visualise the output waveform using the `data_inspector` hierarchy.

In [ ]:
#ol.nco.real_enable()    # Cosine only output
ol.nco.complex_enable() # Cosine and Sine wave output

### 3.2. Widgets <a class="anchor" id="widgets"></a>

If desired, we can directly tie the frequency property to a widget that you can interact with. For instance, the frequency can be controlled using a slider.

In [ ]:
import ipywidgets as ipw

def freq_callback(frequency):
    ol.nco.frequency = frequency

freq_slider = ipw.FloatSlider(min=-50e6, max=50e6, step=1e6, value=12e6)
ipw.interact(freq_callback, frequency=freq_slider);

The gain can also be controlled using a slider widget.

In [ ]:
def gain_callback(gain):
    ol.nco.gain = gain

gain_slider = ipw.FloatSlider(min=-1, max=1, step=0.1, value=0.5)
ipw.interact(gain_callback, gain=gain_slider);

Feel free to modify these sliders in any way you like before progressing to the next section.

## 4. Visualisation and Analysis <a class="anchor" id="visualisation-and-analysis"></a>
The output waveform from the NCO can be transferred and plotted for visualisation and analysis. Before we progress further, let us transfer 1024 samples of data from the FPGA into Jupyter Labs using the `ol.data_inspector.transfer(packetsize)` method of the data inspector hierarchy.

In [ ]:
data = ol.data_inspector.transfer(512)
data[0:16] # First 16 samples

The samples above are using complex notation as the NCO can output a cosine and sine wave at the same time. If you previously chose to use the cosine wave only, then the imaginary part of the complex number will be 0j. If you chose to use the cosine wave and sine wave, then the real part of the complex number is the cosine wave, and the imaginary part of the complex number refers to the sine wave.

We can use the Plotly Graphics Objects library to simply create an interactive plot of NCO data. Import the Plotly library below.

In [ ]:
import plotly.graph_objs as go

### 4.2. Time Plot <a class="anchor" id="time-plot"></a>
Firstly, we will plot the real and imaginary components of the NCO wave below, using the Plotly Graphics Objects Scatter class. The plot may take a moment to initialise.

In [ ]:
go.Figure(data=[go.Scatter(name="Real Data", y=data.real),
                go.Scatter(name="Imag Data", y=data.imag)],
          layout={
              'title' : 'NCO Time Plot',
              'yaxis' : {
                  'title' : 'Amplitude'
              },
              'xaxis' : {
                  'title' : 'Samples (n)'
              }})

You can interact with the plot above. Click and drag over the plot to zoom into a particular region of the waveform. Remember that the imaginary data trace will be zero if you did not select to use the cosine and sine wave output of the NCO.

### 4.3. Frequency Spectra <a class="anchor" id="frequency-spectra"></a>
We can inspect the output of the NCO in the frequency domain by plotting its corresponding log-scale magnitude spectra. We will use the NumPy Fast Fourier Transform (FFT) module to convert the time domain waveform into its frequency domain equivalent (more on the frequency domain in a later notebook).

In [ ]:
import numpy as np

Now, we obtain the log-scale magnitude spectra and plot the results using the Plotly Scatter class.

In [ ]:
X = np.fft.fftshift(np.fft.fft(data))
X_mag = np.abs(X)
X_norm = X_mag/len(X)
X_log = 10*np.where(X_norm>0, np.log10(X_norm), 0)

go.Figure(data=go.Scatter(y=X_log, x=np.arange(-100e6/2, 100e6/2, 100e6/len(X))),
          layout={
              'title' : 'NCO Complex Frequency Spectra',
              'yaxis' : {
                  'title' : 'Log-Scale Magnitude (dB)'
              },
              'xaxis' : {
                  'title' : 'Frequency (Hz)'
              }})

You should be able to observe a peak in the spectra plot above. The peak will be over the top of the NCO frequency you selected previously. If you chose to use the cosine output of the NCO only, then you will see two peaks in the plot. One will be at the positive NCO frequency and the other will be at the negative NCO frequency. A complex NCO output that consists of cosine and sine waves will only have one peak over the NCO frequency.

## 5. Conclusion <a class="anchor" id="conclusion"></a>
This notebook has presented a simple NCO overlay for your PYNQ supported platform. We explored the PYNQ overlay, DefaultIP, and DefaultHierarchy built-in classes. Additionally, we investigated an NCO and configured its phase and gain properties. Lastly, we plotted the output waveform of the NCO in time and frequency domains.

In the next notebook, we explore discrete sampling.

---

[⬅️ Previous Notebook](03_pynq_introduction.ipynb) || [Next Notebook 🚀](../notebook_B/01_sampling.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---